In [3]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pandas as pd
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)

/home/codespace/.python/current/lib/python3.10/site-packages/pytorch_tabular/models/mixture_density/mdn.py:25: UserWarning: Wandb not installed. WandbLogger will not work.
  warnings.warn("Wandb not installed. WandbLogger will not work.")


In [4]:
# Lorenz system
def lorenz63(t, X):
    u, v, w = X
    up = -10*(u - v)
    vp = 28*u - v - u*w
    wp = -2.667*w + u*v
    return up, vp, wp

In [6]:
# Simulation params
# max time, number of data points
tmax, n = 100, 1000  

In [7]:
# PyTorch model
# Lags at 1-2-3-4-5 as feature
data_config = DataConfig(
    target=["x", "y", "z"],
    continuous_cols=['x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 'x_3', 'y_3',
       'z_3', 'x_4', 'y_4', 'z_4', 'x_5', 'y_5', 'z_5'],
)

trainer_config = TrainerConfig(
    auto_lr_find=True,
    batch_size=64,
    max_epochs=100,
)

optimizer_config = OptimizerConfig()

# Three layer MLP with LeakyReLU activation functions
model_config = CategoryEmbeddingModelConfig(
    task="regression",
    layers="32-16-16",
    activation="LeakyReLU",
    learning_rate=1e-3,
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-04-15 17:37:05,760 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off


In [8]:
# Train and predict for 10 different starting conditions
for _ in range(10):
    # Generate random starting point
    u0 = np.random.randn()
    v0 = np.random.randn()
    w0 = np.random.randn()

    # Solve system
    soln = solve_ivp(lorenz63, (0, tmax), (u0, v0, w0), dense_output=True)
    t = np.linspace(0, tmax, n)
    x, y, z = soln.sol(t)
    
    # Prepare dataset
    df = pd.DataFrame(np.concatenate([x.reshape(-1, 1), y.reshape(-1, 1), z.reshape(-1, 1)], axis=1), columns=["x", "y", "z"])
    for i in range(5):
        df[f"x_{i+1}"] = df.x.shift(i+1)
        df[f"y_{i+1}"] = df.y.shift(i+1)
        df[f"z_{i+1}"] = df.z.shift(i+1)
    df.dropna(inplace=True)

    # Split data as train-valid-test
    traindata, validdata, testdata = df.iloc[0:500,:], df.iloc[501:800,:], df.iloc[801:,:]
    
    # Fit - predict
    tabular_model.fit(train=traindata, validation=validdata)
    result = tabular_model.evaluate(testdata)
    pred_df = tabular_model.predict(testdata)
    
    # Plot
    fig, ax = plt.subplots(3, 1)
    ax[0].set_title(f"Starting conditions: {np.round(u0, 6)},{np.round(v0, 6)},{np.round(w0, 6)}")
    ax[0].plot(pred_df['x'], label="x")
    ax[0].plot(pred_df['x_prediction'], label="pred")
    ax[0].legend()
    ax[1].plot(pred_df['y'], label="y")
    ax[1].plot(pred_df['y_prediction'], label="pred")
    ax[1].legend()
    ax[2].plot(pred_df['z'], label="z")
    ax[2].plot(pred_df['z_prediction'], label="pred")
    ax[2].legend()
    plt.show()

Global seed set to 42
2023-04-15 17:38:47,588 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-04-15 17:38:47,590 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for regression task
2023-04-15 17:38:47,632 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: CategoryEmbeddingModel
/home/codespace/.python/current/lib/python3.10/site-packages/pytorch_tabular/models/base_model.py:126: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
2023-04-15 17:38:47,790 - {pytorch_tabular.tabular_model:264} - INFO - Preparing the Trainer
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-04-15 17:38:47,870 - {pytorch_tabular.tabular_model:558} - INFO - Auto LR Find Started


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.04365158322401657
Restoring states from the checkpoint path at /workspaces/pytorch-examples/.lr_find_c69389d8-e2ce-485c-8406-9f4b0d7e7e81.ckpt
Restored all states from the checkpoint file at /workspaces/pytorch-examples/.lr_find_c69389d8-e2ce-485c-8406-9f4b0d7e7e81.ckpt
2023-04-15 17:38:49,584 - {pytorch_tabular.tabular_model:560} - INFO - Suggested LR: 0.04365158322401657. For plot and detailed analysis, use `find_learning_rate` method.
2023-04-15 17:38:49,586 - {pytorch_tabular.tabular_model:566} - INFO - Training Started


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  1.3 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │     30 │
│ 2 │ head             │ LinearHead                │     51 │
│ 3 │ loss             │ MSELoss                   │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 1.4 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.4 K                                                                                                
Total estimated model params size (MB): 0

Output()